In [1]:
import re
import requests
import pandas as pd
from io import StringIO
from FinanceDataReader._utils import (_convert_letter_to_num, _validate_dates)

In [12]:

# HLB 종목코드 : 028300 , 시작일 종료일 지정
url = 'https://fchart.stock.naver.com/sise.nhn?timeframe=day&count=6000&requestType=0&symbol=' # url
r = requests.get(url + "028300") # 종목코드

data_list = re.findall('<item data=\"(.*?)\" />', r.text, re.DOTALL)
data = '\n'.join(data_list)
df = pd.read_csv(StringIO(data), delimiter='|', header=None, dtype={0:str})
df.columns  = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df['Change'] = df['Close'].pct_change(periods=5)
result = df.loc['2020-01-01':'2024-07-01'] # 시작일:종료일
print(result)

             Open   High    Low  Close   Volume    Change
Date                                                     
2020-01-02  46566  46687  45054  45495  2658091  0.055397
2020-01-03  45772  45812  42587  43187  3915498  0.083902
2020-01-06  41512  45891  41074  42311  5247018  0.052486
2020-01-07  42786  44220  42229  43544  2190296  0.022424
2020-01-08  42707  43264  40398  40639  4183583 -0.111405
...           ...    ...    ...    ...      ...       ...
2024-06-25  66100  73800  65900  72800  6153731  0.159236
2024-06-26  72000  72700  60800  65000  6976235  0.025237
2024-06-27  63200  65600  63100  63900  1696469  0.006299
2024-06-28  63800  65600  58100  58500  3770235 -0.090202
2024-07-01  59400  63800  58500  62600  3064043 -0.051515

[1109 rows x 6 columns]


In [13]:
# 주가 5% 이상 10%미만 급등한 경우 (종가 기준)
increasing_5 = result['Change'] >= 0.05
below_10 = result['Change'] < 0.1
num_case_5 = len(result[increasing_5 & below_10])
print(f"# 5% 이상 10%미만 급등한 경우 : {num_case_5}번")
above_5 = result[increasing_5 & below_10]
print(above_5)

# 5% 이상 10%미만 급등한 경우 : 121번
              Open    High     Low   Close   Volume    Change
Date                                                         
2020-01-02   46566   46687   45054   45495  2658091  0.055397
2020-01-03   45772   45812   42587   43187  3915498  0.083902
2020-01-06   41512   45891   41074   42311  5247018  0.052486
2020-01-13   44775   45293   43980   44579  1101404  0.053603
2020-02-06   39163   43064   39004   41594  2808135  0.082895
...            ...     ...     ...     ...      ...       ...
2024-04-19   97300  100000   93000  100000  2060066  0.075269
2024-04-22  103200  109300  101800  106300  3046455  0.079188
2024-06-13   62500   67000   61500   63600  5693263  0.067114
2024-06-17   61500   63600   61000   63200  1209337  0.067568
2024-06-18   63000   64400   62300   62800  1117297  0.073504

[121 rows x 6 columns]


In [14]:
# 주가 10% 이상 급등한 경우 (종가 기준)
increasing_10 = result['Change'] >= 0.1
num_case_10 = len(result[increasing_10])
print(f"# 10% 이상 급등한 경우 : {num_case_10}번")
above_10 = result[increasing_10]
print(above_10)

# 10% 이상 급등한 경우 : 125번
             Open   High    Low  Close    Volume    Change
Date                                                      
2020-01-15  44617  45492  44220  45137   1524930  0.110682
2020-02-07  41591  42148  40477  40838   1032360  0.106781
2020-03-02  38807  39643  37732  38490   1824431  0.117920
2020-04-02  37214  38686  36140  38331   4221864  0.103146
2020-04-03  38806  39522  37611  38450   3125048  0.129852
...           ...    ...    ...    ...       ...       ...
2024-05-28  56700  67600  55700  64700  17559460  0.334021
2024-05-29  63000  66900  62600  66100   6560217  0.340771
2024-05-30  65800  66600  61200  61500   5062564  0.262834
2024-05-31  62500  64800  58300  61200   5754591  0.207101
2024-06-25  66100  73800  65900  72800   6153731  0.159236

[125 rows x 6 columns]


In [5]:
result.to_csv("HLB_price(20.1~24.7).csv", index= True)

In [6]:
above_5.to_csv("HLB_above_5.csv", index=True)
above_10.to_csv("HLB_above_10.csv", index=True)